In [3]:
pip install -r requirements.txt

  Using cached pandas-2.0.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.2 MB)
  Using cached matplotlib-3.7.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.6 MB)
  Using cached scikit_learn-1.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.6 MB)
  Using cached tensorflow-2.12.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (586.0 MB)
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached contourpy-1.0.7-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (299 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached fonttools-4.39.4-py3-none-any.whl (1.0 MB)
  Using cached kiwisolver-1.4.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.4 MB)
  Using cached Pillow-9.5.0-cp311-cp311-manylinux_2_28_x86_64.whl (3.4 M

In [6]:
import os
import glob
import numpy as np
import scipy.io
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as K
from sklearn import svm
from sklearn.preprocessing import StandardScaler
import tensorflow_addons as tfa
import pandas as pd
import matplotlib.pyplot as plt
adam = tf.keras.optimizers.Adam(learning_rate=0.0005)
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Input, Conv1D, MaxPooling1D, Flatten, Dense
import numpy as np
import scipy.io as sio
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Reshape
temperature = 0.03
learning_rate=0.001

2023-05-23 11:21:49.074663: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-23 11:21:49.181174: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-23 11:21:49.181873: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 11:21:50.499233: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in M

In [7]:
def dataloader(path, featType):
    """
    Load data from a MATLAB file.

    Args:
        path (str): Path to the MATLAB file.
        featType (int): Type of features to load.

    Returns:
        Tuple: Tuple containing input features, labels, weights, and additional information.
    """
    data = scipy.io.loadmat(path)
    print(data.keys())

    AF = data['AF']
    x1 = AF[:-2]
    y = AF[-2]
    w = AF[-1]

    if featType == 1:
        x = x1
    else:
        x2 = data['CF']
        x = np.concatenate((x1, x2), axis=0)
    return x.T, y.T, w.T, data['CF_info']

def calculate_accuracy(arr1, arr2):
    """
    Calculate the accuracy between two arrays.

    Args:
        arr1 (array): First array.
        arr2 (array): Second array.

    Returns:
        float: Accuracy between the two arrays.
    """
    count = sum(1 for itr1, itr2 in zip(arr1, arr2) if itr1 == itr2)
    return count / len(arr1)

def normalization(feats):

    """
    Normalize the input features using standard scaling.

    Args:
        feats (array): Input features.

    Returns:
        array: Normalized features.
    """
    df = pd.DataFrame(feats)
    scaler = StandardScaler()
    x_new = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    return x_new

def make_partitions(arr_words, arr_labels):

    """
    Create partitions based on word boundaries and labels.

    Args:
        arr_words (array): Array of words.
        arr_labels (array): Array of labels.

    Returns:
        array: Partitions based on word boundaries and labels.
    """
    v = []
    temp = []

    for i in range(len(arr_words) - 1):
        word = arr_words[i]
        next_word = arr_words[i + 1]
        temp.append(arr_labels[i])

        if word != next_word or i == len(arr_words) - 2:
            if i == len(arr_words) - 2:
                temp.append(arr_labels[i + 1])

            numpy_temp = np.array(temp)
            temp_max = np.amax(numpy_temp)
            numpy_temp = np.divide(numpy_temp, temp_max)
            v = np.concatenate((v, numpy_temp), axis=None)
            temp.clear()

    v1 = [1 if i == 1 else 0 for i in v]
    return v1

In [8]:
fatyp = 'TypicalFA_comb1'
drivepath = 'finalData/'+ fatyp +'/';
#featFiles = 'GER_train_fisher-2000_FA_GT_ESTphnTrans_estStress'               #glob.glob(drivepath + '*train*')
filee = drivepath+'GER_train_fisher-2000_FA_GT_ESTphnTrans_estStress.mat'
featType = 1; #Acoustic or Acoustic+context
if featType == 1:
  original_dim = 19
else:
  original_dim = 38

In [9]:
# print('Classification with::::::',os.path.basename(filee))

train_path = filee; test_path = filee.replace('train','test')
# print('test file:::::::',os.path.basename(test_path))
xtrain, ytrain, wtrain, info_train = dataloader(train_path, featType); 
xtrain1, ytrain1, wtrain1, info_train1 = dataloader(train_path, featType=2);
xtest, ytest, wtest ,info_test = dataloader(test_path, featType)
xtest1, ytest1, wtest1, info_test1 = dataloader(test_path, featType=2);



dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])
dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])
dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])
dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])


In [10]:
xtest_a = normalization(xtest)
xtest_ac = normalization(xtest1)
xtrain = normalization(xtrain)
xtrain1 = normalization(xtrain1)

woPP=[]; wPP=[]
input_shape1 = (19,1)
input_shape2 = (38,1)
temperature = 0.03
learning_rate=0.001

In [11]:
class SupervisedContrastiveLoss(keras.losses.Loss):
    def __init__(self, temperature=temperature, name=None):
        super(SupervisedContrastiveLoss, self).__init__(name=name)
        self.temperature = temperature

    def __call__(self, labels, feature_vectors, sample_weight=None):
        # Normalize feature vectors
        print(feature_vectors.shape)
        # labels = tf.keras.layers.Concatenate(axis=0)([labels, labels])
        feature_vectors_normalized = tf.math.l2_normalize(feature_vectors, axis=1)

        # Compute logits
        logits = tf.divide(tf.matmul(feature_vectors_normalized, tf.transpose(feature_vectors_normalized)), self.temperature)
        
        # print(feature_vectors.shape)
        # print(labels.shape)
        # print('loss:::::::', tfa.losses.npairs_loss(tf.squeeze(labels), logits))
        return 0.35*(tfa.losses.npairs_loss(tf.squeeze(labels), logits))
        #find out more about why 0.35 is used

In [12]:
latent_dim = 30

class Encoder1(Model):
  def __init__(self, latent_dim):
    super(Encoder1, self).__init__()
    self.latent_dim = latent_dim 
    inputs = Input(shape=(19,1))
    outputs = inputs  
    self.encoder = tf.keras.Sequential([
      inputs,
      
      layers.Conv1D(filters=64, kernel_size=2, activation='relu'),
      layers.MaxPooling1D(pool_size=1),
      layers.Dropout(0.5),
      layers.Conv1D(filters=32, kernel_size=2, activation='relu'),
      layers.MaxPooling1D(pool_size=1),
      layers.Flatten(),
      layers.Dense(latent_dim, activation='relu'),
    ])

  def call(self, x):
    encoded = self.encoder(x)
    return encoded

class Encoder2(Model):
  def __init__(self, latent_dim):
    super(Encoder2, self).__init__()
    self.latent_dim = latent_dim
    inputs = Input(shape=(38,1))
    outputs = inputs  
    self.encoder = tf.keras.Sequential([
      inputs,
      layers.Conv1D(filters=64, kernel_size=2, activation='relu'),
      layers.MaxPooling1D(pool_size=1),
      layers.Dropout(0.5),
      layers.Conv1D(filters=32, kernel_size=2, activation='relu'),
      layers.MaxPooling1D(pool_size=1),
      layers.Flatten(),
      layers.Dense(latent_dim, activation='relu'),
    ])

  def call(self, x):
    encoded = self.encoder(x)
    return encoded

In [13]:
def create_encoder1():
    return Encoder1(latent_dim)

def create_encoder2():
    return Encoder2(latent_dim)

def add_projection_head1(Encoder1, Encoder2):
    inp1 = keras.Input(shape=input_shape1)
    inp2 = keras.Input(shape=input_shape2)
    hidden3a  = Encoder1(inp1)
    hidden3b = Encoder2(inp2)
    features = tf.keras.layers.Concatenate(axis=1)([hidden3a, hidden3b])
    features = layers.Dense(16, activation="relu")(features)
    model = keras.Model(
        inputs=[inp1, inp2], outputs=features, name="cifar-encoder_with_projection-head"
    )
    return model

In [14]:
def create_classifier(encoder, trainable):

    for layer in encoder.layers:
        layer.trainable = trainable

    inputs1 = keras.Input(shape=input_shape1)
    inputs2 = keras.Input(shape=input_shape2)
    features1 = encoder1(inputs1)
    features2 = encoder2(inputs2)
    features = tf.keras.layers.Concatenate(axis=1)([features1, features2])
    # features = layers.Dropout(0.2)(features)
    features = layers.Dense(hidden_units, activation="relu")(features)
    
    features = layers.BatchNormalization()(features)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(32, activation="relu")(features)
    features = layers.Dropout(0.2)(features)
    features = layers.Dense(16, activation="relu")(features)
    # features = layers.BatchNormalization()(features)
    # features = layers.Dropout(0.2)(features)
    features = layers.Dense(4, activation="relu")(features)
    # features = layers.BatchNormalization()(features)
    # features = layers.Dropout(0.1)(features)
    outputs = layers.Dense(num_classes, activation="sigmoid")(features)

    model = keras.Model(inputs=[inputs1,inputs2], outputs=outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model

#single iteration

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
# Splitting xtrain and ytrain into training and validation sets
xtra_a, xval_a, ytra_a, yval_a = train_test_split(xtrain, ytrain, test_size=0.2, random_state=42)

# Splitting xtrain1 and ytrain1 into training and validation sets
xtra_ac, xval_ac, ytra_ac, yval_ac = train_test_split(xtrain1, ytrain1, test_size=0.2, random_state=42)

In [16]:
y = np.concatenate((ytra_a,ytra_a), axis=0)
yv = np.concatenate((yval_a,yval_a), axis=0)

encoder1 = create_encoder1()
encoder2 = create_encoder2()
encoder_with_projection_head = add_projection_head1(encoder1, encoder2)
encoder_with_projection_head.compile(optimizer=keras.optimizers.Adam(learning_rate),loss=SupervisedContrastiveLoss(temperature))
encoder_with_projection_head.summary()
                                                            #ytra_a                                  #yval_a
history = encoder_with_projection_head.fit([xtra_a,xtra_ac], ytra_ac , validation_data =([xval_a,xval_ac],yval_ac), batch_size=32, epochs=100, callbacks=[EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)])


learning_rate = 0.0005
batch_size = 16
hidden_units = 64
projection_units = 128
num_epochs = 200
dropout_rate = 0.3
num_classes = 2
input_shape1 = (19,)
input_shape2 = (38,)

from keras.callbacks import ModelCheckpoint ,EarlyStopping
classifier = create_classifier(encoder_with_projection_head, trainable=False)
classifier.summary()
history = classifier.fit(x=[xtra_a,xtra_ac], y=ytra_a, validation_data =([xval_a,xval_ac],yval_a), batch_size=batch_size, epochs=num_epochs, callbacks=[EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)])

accuracy = classifier.evaluate([xtest_a,xtest_ac], ytest)[1]

##  Accuracy on post processed labels (each word should contain only one stressed syllable)
pred_output= classifier.predict([xtest_a,xtest_ac])
# pred_labels= pred_output.argmax(axis =1)
pred1_labels = pred_output[:,1]
post_labels = make_partitions(wtest, pred1_labels)
post_accuracy = calculate_accuracy(post_labels, ytest)

F1_score_WoPP = f1_score(ytest, pred_output.argmax(axis =1))
F1_score_WPP = f1_score(ytest, post_labels)

print(f"Test accuracy: {round(accuracy * 100, 3)}%")
print(f"Postprocessing Test accuracy: {round(post_accuracy * 100, 3)}%")
print(f"Test F1_score: {round(F1_score_WoPP * 100, 3)}%")
print(f"Postprocessing F1_score: {round(F1_score_WPP * 100, 3)}%")

Model: "cifar-encoder_with_projection-head"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 19, 1)]      0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 38, 1)]      0           []                               
                                                                                                  
 encoder1 (Encoder1)            (None, 30)           20670       ['input_3[0][0]']                
                                                                                                  
 encoder2 (Encoder2)            (None, 30)           38910       ['input_4[0][0]']                
                                                                 